# SSD Train

In [1]:
import os
import cv2
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

import pickle
import random

from ssd.ssd import SSD
from ssd.ssd_loss import MultiboxLoss
from ssd.ssd_bbox import BBoxUtility

%matplotlib inline
plt.rcParams['figure.figsize'] = (8, 8)
plt.rcParams['image.interpolation'] = 'nearest'

np.set_printoptions(suppress=True)

# config = tf.ConfigProto()
# config.gpu_options.per_process_gpu_memory_fraction = 0.9
# set_session(tf.Session(config=config))

# Notebook auto reloads code. (Ref: http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython)
% load_ext autoreload
% autoreload 2

### load data

In [2]:
# some constants
NUM_CLASSES = 4 # remember the background 
input_shape = (300, 300, 3)

In [7]:
"""
tf.reset_default_graph()
## priorbox defined by ourselves
prior_tensor = tf.ones((1, 300, 300, 3))
ssd_net = SSD(input_shape, NUM_CLASSES)
_, priors = ssd_net(prior_tensor, PRIORS = True)
print(priors.shape)
"""

(?, 7308, 8)


In [4]:
# DEBUG by tensorboard
"""
g = tf.Graph()
with g.as_default():
    #a = tf.placeholder(tf.float32, name="a")
    #b = tf.placeholder(tf.float32, name="b")
    #c = a + b
    ## priorbox defined by ourselves
    prior_tensor = tf.ones((1, 300, 300, 3))
    ssd_net = SSD(input_shape, NUM_CLASSES)
    _, priors = ssd_net(prior_tensor, PRIORS = True)
    print(priors.shape)
    
tf.summary.FileWriter("logs", g).close()
"""

'\ng = tf.Graph()\nwith g.as_default():\n    #a = tf.placeholder(tf.float32, name="a")\n    #b = tf.placeholder(tf.float32, name="b")\n    #c = a + b\n    ## priorbox defined by ourselves\n    prior_tensor = tf.ones((1, 300, 300, 3))\n    ssd_net = SSD(input_shape, NUM_CLASSES)\n    _, priors = ssd_net(prior_tensor, PRIORS = True)\n    print(priors.shape)\n    \ntf.summary.FileWriter("logs", g).close()\n'

In [9]:
# save priors tensor
#with open('./data/priors300.pkl', 'wb') as handle:
#    pickle.dump(priors_np, handle)

In [3]:
# load priors tensor
with open('./data/priors300.pkl', 'rb') as handle:
    priors_np = pickle.load(handle)
    print(priors_np.shape)

(7308, 8)


### bbox

In [4]:
#sess = tf.InteractiveSession()
#sess.run(tf.global_variables_initializer())
#priors_np = np.squeeze(sess.run(priors))
bbox_util = BBoxUtility(NUM_CLASSES, priors_np)

In [5]:
print([None,] + list(priors_np.shape))

[None, 7308, 8]


### data

In [6]:
## bounding box ground truth
gt = pickle.load(open('./data/indata.pkl', 'rb'))
keys = sorted(gt.keys())
random.shuffle(keys)
num_train = int(round(0.8 * len(keys)))
train_keys = keys[:num_train]
val_keys = keys[num_train:]
num_val = len(val_keys)
print(num_train)
print(num_val)

333
83


In [7]:
from ssd.preprocess import *
## input images
## define input data generator: return a batch of input images as well as target feature vectors 
## like 64587, 8+num_classes
path_prefix = './data/train/'

# batch_size = 4
gen = Generator(gt, bbox_util, 4, './data/train/',
                train_keys, val_keys,
                (input_shape[0], input_shape[1]), do_crop=False)

Using TensorFlow backend.


### build model

In [8]:
batch_size = 4

In [9]:
tf.reset_default_graph()
input_ph = tf.placeholder(tf.float32, [None,] + list(input_shape), name="input_image")
gt_ph = tf.placeholder(tf.float32, [None, 7308, 16], name="ground_truth")
with tf.variable_scope("ssd_net"):
    ssd_net = SSD(input_shape, NUM_CLASSES)
    predictions = ssd_net(input_ph, PRIORS = False)
    
with tf.variable_scope("loss"):
    loss = MultiboxLoss(NUM_CLASSES, neg_pos_ratio=2.0).compute_loss(gt_ph, predictions)
    
with tf.variable_scope("train"):
    global_step = tf.Variable(0, trainable=False)
    lr = tf.train.exponential_decay(3e-4, global_step, num_train//batch_size, 0.9, staircase=True, name="lr")
    train_op = tf.train.AdamOptimizer(learning_rate=3e-4).minimize(loss, global_step=global_step)

/Users/tingyumao/Programs/anaconda3/envs/py35/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


### train

In [ ]:
nb_epoch = 20
train_loss_hist = []

train_generator = gen.generate(True)
val_generator = gen.generate(False)

sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
for e in range(nb_epoch):
    # training phase
    cnt = 0
    for sample in train_generator:
        _, train_input, train_target = sample
        train_loss, _ = sess.run([loss, train_op], feed_dict={input_ph: train_input, gt_ph: train_target})
        train_loss_hist.append(train_loss)
        cnt += 1
        print("training {}/{}, minibatch loss: {}".format(cnt*batch_size, num_train, train_loss))
        if cnt > (num_train//batch_size+1):
            cnt = 0
            break
        
    # validation phase
    val_loss = 0.0
    for sample in val_generator:
        _, val_input, val_target = sample
        val_loss += sess.run(loss, feed_dict={input_ph: val_input, gt_ph: val_target})
        if cnt > (num_val//batch_size+1):
            cnt = 0
            break
    
    # print progress
    verbose_loss = np.mean(train_loss_hist[-(num_train//batch_size+1):])
    print("epoch {}/{}, train loss: {}, val loss: {}".format(e, nb_epoch, verbose_loss, val_loss))

training 4/333, minibatch loss: [ 6.46188307  5.89722157  6.77153778  5.8900547 ]
training 8/333, minibatch loss: [ 18.93037033  16.27460098  24.52888489  17.32169914]
training 12/333, minibatch loss: [  9.07914543   8.41857529  10.43834972   9.04242897]
training 16/333, minibatch loss: [ 4.99908066  4.07078409  5.23704863  5.61124802]
training 20/333, minibatch loss: [ 5.92088461  5.40038872  7.35792685  6.34349632]
training 24/333, minibatch loss: [ 5.9510498   5.00807667  5.46769619  4.48874092]
training 28/333, minibatch loss: [ 3.90168166  3.9039278   3.96347284  3.61905575]


### evaluate

In [43]:
gen.generate(True)

<generator object Generator.generate at 0x1419b9728>

In [48]:
train_generator = gen.generate(True)